In [ ]:
%%time
print("start")
%run -i 'myimports.py'
# %run -i "youtube_extraction.py"
# %run -i "nlp_processing.py"
%run -i "timestamp_generation"
%run -i "datagathering"


#get collection of documents (minute of podcast) in a list
main="https://www.happyscribe.com/public/lex-fridman-podcast-artificial-intelligence-ai/"
transfolder ="3Lex/"
timesfolder="lextimestamps2/"
url="101-joscha-bach-artificial-consciousness-and-the-nature-of-reality"
file = "#101|Joscha_Bach|Artificial_Consciousness_and_the_Nature_of_Reality.txt"
write_specific(main, url, transfolder)
document_size = 1000
topic_size = 2000

soup = BeautifulSoup(requests.get(main+url).text,"html.parser")
sents = list(map(contractions.fix, re.split("(?<=[.?!])\s", open(transfolder+file).read())))

dictionary, model = generate_model(transfolder+file, document_size, topic_size)
one_topic_confi = load_confidences(transfolder+file, topic_size, dictionary, model, sents, basic_completion) #generate initial topics + confidences, then smooth by filling in empty values and averaging


algo_stamp = get_algo_timestamps(one_topic_confi) #algo generated timestamps
final_topic_confi = get_final_topic_confi(sents, algo_stamp) #new topic graph based on algo timestamps


actual_stamps = get_real_timestamps(soup, sents, timesfolder, file) #description generated timestamps
# timestamp_array = convert_topics_to_graph(actual_stamps, sents) #covert discription timestamp to graphable array (black bars)

 #get top topic predictions for each sentence
top_confi = np.zeros([int(pod_word_count(transfolder+file)/topic_size), len(sents)])



In [ ]:
print("Description Timestamps:", len(actual_stamps))
print("Algo Timestamps:", len(algo_stamp))

In [ ]:
NUM_COLORS = int(pod_word_count(transfolder+file)/topic_size)
top_confi = np.zeros([int(pod_word_count(transfolder+file)/topic_size), len(sents)])
cm = plt.get_cmap('gist_rainbow')
fig = plt.figure()
ax = fig.add_subplot(111)
ax.set_prop_cycle('color', [cm(1.*i/NUM_COLORS) for i in range(NUM_COLORS)])
plt.title("Topic Confidences")
plt.xlabel("Sentence Number")
plt.xticks(np.arange(0, len(sents), 50))
plt.xlabel("Confidence Level")
plt.ylim(0,1)

r1, r2 = 0, len(sents)
# r1, r2 = 150, 200
x = np.arange(0, len(sents))
plt.plot(x[r1:r2], convert_to_gra(actual_stamps, sents)[r1:r2], label="Real Timestamps", color="black") #graph original timestamps in black
# plt.plot(x[r1:r2], final_topic_confi[r1:r2], label="Model", color="dimgrey")



for ind, i in enumerate(final_topic_confi): #set graph
    top_confi[int(i)][ind] = .8
    
for i in range(int(pod_word_count(transfolder+file)/topic_size)): #graph the timestamps as according to algorithm, each color corresponds to a new topic
    plt.plot(x[r1:r2],top_confi[i][r1:r2], label="Topic: "+str(i))

plt.legend()
plt.rcParams['figure.figsize'] = [40, 10]
plt.show()

In [ ]:
#manually obtained timestamp sentences

# breaks = ["So can we try to reverse engineer your mind a little bit?",
#          "So what is your sense in the philosophical?",
#          "Do you think of computation and computer science and you kind of represent that to me?",
#          "Can you talk about what is dualism or does idealism or is materialism?",
#           "But why does the simulation feel like something?",
#           "But what is the connection between the the brain, the mind and the broader universe which comes first, which is more fundamental?",
#           "So have you ever seen artificial intelligence or any kind of construction idea that allows for unlike neural networks or perhaps within neural networks, that is able to form something where the space of concepts continues to be integrated?",
#           "What kind of mechanisms might we move towards to improve the learning procedure?",
#           "or so that you discovered this?",
#           "The dopamine rush of the like, no matter how like I consider I tried to be very kind of Zen like and minimalist and not be influenced by likes and so on.",
#           "What kind of mechanisms of government do you think we as a species should be evolving with us?",
#           "So what is the trajectory out of self-destruction, do you think?",
#           "What is the most beautiful to you, silly ape, the most beautiful, surprising idea in the development of artificial intelligence, whether in your own life or in the history of artificial intelligence that you have come across.",
#           "All the competitors in the game are improving gradually.",
#           "Do you think that ajai systems that we try to create or greater intelligence systems would need to have a body?",
#           "How does that change the fundamentals of the zeroth order in the first order?",
#           "If you can directly access all of Wikipedia in trying to understand what is the best form of government, then every thought will not be stuck in a like every thought that requires some extra piece of information will be able to grab it really quickly.",
#           "But in the context of the mind, what is the role of emotion and what is it?",
#           "So do you think suffering is fundamental to happiness along these lines, suffering as the result of caring about things that you cannot change?Z"        
#          ]


# timestamp_array = np.zeros([len(sents), 1])
# for ind, sent in enumerate(sents):
#     if next((True for single in breaks if sent in single), False):
#         print(ind)
#         timestamp_array[ind] =1

In [ ]:
#extra function to print places with some confusion
# window = 7
# toskip = 0
# training_sets = [[i] for i in range(window*2)]
# for ind, i in enumerate(one_topic_confi):
#     if toskip>0:
#         toskip-=1
#         continue
#     if ind>window and ind<len(one_topic_confi)-window: #iterate through deeper windows in sntence topic list buyt not too deep
#         if i[0] ==-1 or i[0]!=stats.mode(one_topic_confi[max(-window+ind, 0):min(ind+window, len(one_topic_confi))][:,0])[0]: #not properly classified points
#             datapt = one_topic_confi[max(-window+ind, 0):min(ind+window, len(one_topic_confi))][:,0] #current window
#             if len(set(datapt)) >2: #if a window contians more than 2 points
#                 mode1 = stats.mode(datapt)[0]
#                 mode2 = stats.mode([i for i in datapt if i!=stats.mode(datapt)[0]])[0]
#                 print(ind, datapt, stats.mode(datapt)[0])
#                 mask = np.empty(len(datapt))
#                 for ind, i in enumerate(datapt):
#                     if i!=mode1 and i!=mode2:
#                         mask[ind] = -1
#                     else:
#                         mask[ind] = i